# $N$-QPU VQE - Horizontal Speedup

This notebook implements routines for running the same VQE ansatz on several Quantum Computers, thus achieving a horizontal speedup, using Interlin-q. Here, the controller host partitions the different observables of the Hamiltonian, along with their coefficients, among the computing hosts. Then, it sends out the ansatz. Finally, it asks for the expectation value later, and reduce the values to a single expectation value through summation. This approach should scale up with the increasing number of computing hosts.

In this notebook, we will be applying VQE for the Hamiltonian of the Hydrogen molecule, which requires a maximum of 4 qubits for its ansatz, which is displayed below:

<img src="https://pennylane.ai/qml/_images/sketch_circuit.png" alt="H2 ansatz" style="width:700px;"/>

## Step 1: Import libraries.

First we import all the necessary libraries. Interlin-q is built using the Python framework [QuNetSim](https://arxiv.org/abs/2003.06397) which is a software framework for simulating quantum networks up to the network layer. We also need PennyLane's chemistry library for decomposing the Hamiltonian.

In [1]:
# Basic Libraries
import sys
import numpy as np
sys.path.append("../../")

# QuNetSim Components
from qunetsim.components import Network
from qunetsim.objects import Logger
from qunetsim.backends.eqsn_backend import EQSNBackend

# Interlin-q Components
from interlinq import (ControllerHost, Constants, Clock,
Circuit, Layer, ComputingHost, Operation)

# Extra needed components
from hamiltonian_decomposition import decompose

Logger.DISABLED = False

In [2]:
import pennylane as qml
qml.version() # should be 0.15.1. Higher versions are not integrated with the notebook yet

'0.15.1'

## Step 2: Decompose the Hamiltonian.

In [3]:
# These parameters are mentioned in PennyLane's VQE tutorial (https://pennylane.ai/qml/demos/tutorial_vqe.html) 
# and are used as is for benchmarking
geometry = 'h2.xyz'
charge = 0
multiplicity = 1
basis_set = 'sto-3g'
name = 'h2'

In [4]:
# The decompose function runs PennyLane's decomposers and strips out the observables from their PennyLane objects
# to be used easily for our purposes
coefficients, observables, qubit_num = decompose(name, geometry, charge, multiplicity, basis_set)
terms = list(zip(coefficients, observables))

terms

[(-0.04207897647782276, [('Identity', 0)]),
 (0.17771287465139946, [('PauliZ', 0)]),
 (0.1777128746513994, [('PauliZ', 1)]),
 (-0.24274280513140462, [('PauliZ', 2)]),
 (-0.24274280513140462, [('PauliZ', 3)]),
 (0.17059738328801052, [('PauliZ', 0), ('PauliZ', 1)]),
 (0.04475014401535161,
  [('PauliY', 0), ('PauliX', 1), ('PauliX', 2), ('PauliY', 3)]),
 (-0.04475014401535161,
  [('PauliY', 0), ('PauliY', 1), ('PauliX', 2), ('PauliX', 3)]),
 (-0.04475014401535161,
  [('PauliX', 0), ('PauliX', 1), ('PauliY', 2), ('PauliY', 3)]),
 (0.04475014401535161,
  [('PauliX', 0), ('PauliY', 1), ('PauliY', 2), ('PauliX', 3)]),
 (0.12293305056183798, [('PauliZ', 0), ('PauliZ', 2)]),
 (0.1676831945771896, [('PauliZ', 0), ('PauliZ', 3)]),
 (0.1676831945771896, [('PauliZ', 1), ('PauliZ', 2)]),
 (0.12293305056183798, [('PauliZ', 1), ('PauliZ', 3)]),
 (0.17627640804319591, [('PauliZ', 2), ('PauliZ', 3)])]

## Step 3: Prepare Circuit for Given Parameters

The circuit can be prepared using two different ways: either as one circuit, or several circuits run sequentially. The former approach is simpler and generally better for the optimisation function. The latter is better for debugging and for dynamic components of a quantum circuit (i.e. circuits that have a lot of changing operations). For the current demonstration, we will use the latter approach, while the former will be used for the optimization part later on.

### Main Blocks

In [5]:
# Arbitrary single qubit rotation, as implemented by PennyLane here: 
# https://pennylane.readthedocs.io/en/stable/code/api/pennylane.Rot.html#pennylane.Rot
def rotational_gate(params):
    phi, theta, omega = params
    cos = np.cos(theta / 2)
    sin = np.sin(theta / 2)
    
    res = np.array([[np.exp(-1j * (phi + omega) / 2) * cos, -np.exp(1j * (phi - omega) / 2) * sin], 
                     [np.exp(-1j * (phi - omega) / 2) * sin, np.exp(1j * (phi + omega) / 2) * cos]])
    
    return res

In [6]:
# These operations are responsible for preparing the initial state of the qubits, as shown in the figure above.
def initialisation_operations(q_map):
    computing_host_ids = list(q_map.keys())
    
    ops = []
    for host_id in computing_host_ids:
        # We first initialize all the qubits in the backend
        op = Operation(
            name=Constants.PREPARE_QUBITS,
            qids=q_map[host_id],
            computing_host_ids=[host_id])
        ops.append(op)
        
        ### Workaround for a bug in EQSN
        op = Operation(
            name=Constants.TWO_QUBIT,
            qids=[q_map[host_id][0], q_map[host_id][1]],
            gate=Operation.CNOT,
            computing_host_ids=[host_id])
        ops.append(op)

        op = Operation(
            name=Constants.TWO_QUBIT,
            qids=[q_map[host_id][0], q_map[host_id][2]],
            gate=Operation.CNOT,
            computing_host_ids=[host_id])
        ops.append(op)

        op = Operation(
            name=Constants.TWO_QUBIT,
            qids=[q_map[host_id][0], q_map[host_id][3]],
            gate=Operation.CNOT,
            computing_host_ids=[host_id])
        ops.append(op)
        ################################
    
        # Prepare the qubits on the computing host, applying X gate to wires 0 and 1
        op = Operation(
            name=Constants.SINGLE,
            qids=[q_map[host_id][0]],
            gate=Operation.X,
            computing_host_ids=[host_id])
        ops.append(op)

        op = Operation(
            name=Constants.SINGLE,
            qids=[q_map[host_id][1]],
            gate=Operation.X,
            computing_host_ids=[host_id])
        ops.append(op)
    
    return [Layer(ops)]

In [7]:
# These operations are responsible for applying the rotation gates on the qubits, as well as the CNOTs.
def ansatz_operations(q_map, parameters):
    computing_host_ids = list(q_map.keys())
    
    layers = []
    
    ops = []
    for host_id in computing_host_ids:
        for i in range(len(q_map[host_id])):
            op = Operation(
                name=Constants.SINGLE,
                qids=[q_map[host_id][i]],
                gate=Operation.CUSTOM,
                gate_param=rotational_gate(parameters[i]),
                computing_host_ids=[host_id])

            ops.append(op)

    layers.append(Layer(ops))
    
    ops = []
    for host_id in computing_host_ids:
        op = Operation(
            name=Constants.TWO_QUBIT,
            qids=[q_map[host_id][2], q_map[host_id][3]],
            gate=Operation.CNOT,
            computing_host_ids=[host_id])
        
        ops.append(op)

        op = Operation(
            name=Constants.TWO_QUBIT,
            qids=[q_map[host_id][2], q_map[host_id][0]],
            gate=Operation.CNOT,
            computing_host_ids=[host_id])

        ops.append(op)

        op = Operation(
            name=Constants.TWO_QUBIT,
            qids=[q_map[host_id][3], q_map[host_id][1]],
            gate=Operation.CNOT,
            computing_host_ids=[host_id])

        ops.append(op)
    
    layers.append(Layer(ops))
        
    return layers

In [8]:
# Send the assigned schedules to the computing hosts
def dispatch_hamiltonian_schedules(host, q_map):        

        # Create the correct operations
        ops = []
        computing_hosts_ids = list(q_map.keys())
        
        for computing_host_id in computing_hosts_ids:    
            op = Operation(
                name=Constants.REC_HAMILTON,
                computing_host_ids=[computing_host_id],
                hamiltonian=host.term_assignment[computing_host_id])
            ops.append(op)

        layers = [Layer(ops)]

        return layers

In [9]:
# Query the computing hosts for their expectation values
def ask_for_expectation_values(q_map):
    
    # Create the correct operations
    ops = []
    computing_hosts_ids = list(q_map.keys())

    for computing_host_id in computing_hosts_ids:    
        op = Operation(
            name=Constants.SEND_EXP,
            computing_host_ids=[computing_host_id])
        ops.append(op)

    layers = [Layer(ops)]

    return layers

### The Protocols

#### The Controller Protocols 

This function builds the complete circuit needed to carry out VQE

In [10]:
def prepare_qubits_and_apply_ansatz(q_map, parameters): 
    circuit = Circuit(q_map, initialisation_operations(q_map) + ansatz_operations(q_map, parameters))
    return circuit

This function details the first communication protocol that will be carried out by the ControllerHost
As shown from the function names, the host first schedules the observables internally, then sends out the circuit to all the ComputingHosts. Note that we didn't send the schedules as of just yet.

In [11]:
def controller_host_protocol_preparation_ansatz(host, q_map, params, terms):
    """
    Protocol for the controller host
    """
    host.schedule_expectation_terms(terms, q_map)
    
    host.generate_and_send_schedules(prepare_qubits_and_apply_ansatz(q_map, params))

Here the ControllerHost builds a schedule, telling the different ComputingHosts to preparing for receiving their part of the Hamiltonian, before sending them right afterwards.

In [12]:
def controller_host_protocol_expectation_schedule(host, q_map):
    """
    Protocol for the controller host
    """
    circuit = Circuit(q_map, dispatch_hamiltonian_schedules(host, q_map))
    
    host.generate_and_send_schedules(circuit)

Here the ControllerHost finally asks for the expectation values back from the ComputingHosts.
Since this is a simulation, the ComputingHosts simply compute their local statevector and calculates the expectation value exactly.

In [13]:
def controller_host_protocol_expectation_collection(host, q_map):
    """
    Protocol for the controller host
    """
    circuit = Circuit(q_map, ask_for_expectation_values(q_map))
    
    host.generate_and_send_schedules(circuit)
    
    host.receive_results()

#### The Computing Protocols

This is the simple communication protocol of the ComputingHosts where they simply receive a schedule of operations and later listen to the synchronization until it is time to run a specific operation (which can a gate or some communication with another party)

In [14]:
def computing_host_protocol(host):
    host.receive_schedule()

In [15]:
# Similar to above, but includes an extra bit for sending the results in the end.
def computing_host_protocol_send_results(host):
    host.receive_schedule()
    
    host.send_results('expectation')

## Step 4: Run the circuit and get the Expectation Value

Let's now run all the different pieces from above and see if we get the expected expectation value, which is the value from the PennyLane tutorial.

In [16]:
def init_network():
    # Retrieve the QuNetSim network
    network = Network.get_instance()
    network.delay = 0
    network.start()

    # Initialize the synchronization clock
    clock = Clock.get_instance()

    # Initialize the statevector backend
    eqsn = EQSNBackend()

    # Initialize the controller host with the given ID
    controller_host = ControllerHost(
        host_id="host_1",
        backend=eqsn
    )

    # Create a network with the given number of QPUs, each with the given number of qubits
    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=2, # This value can be changed per wish. Just make sure you have enough memory!
        num_qubits_per_host=4)
    controller_host.start()

    # Add all the nodes to the QuNetSim network
    network.add_hosts([controller_host])
    network.add_hosts(computing_hosts)
    
    return clock, controller_host, computing_hosts, q_map

In [17]:
# Randomise the ansatz parameters
np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

In [18]:
params

array([[ 5.54193389,  1.25713095,  3.07479606],
       [ 7.03997361,  5.86710646, -3.07020901],
       [ 2.98479079, -0.47550269, -0.32427159],
       [ 1.28993324,  0.45252622,  4.56873497]])

### Running the circuit

In [19]:
# Make sure that the QuNetSim network has no nodes that can interfere with our operation
list_hosts = list(Network.get_instance().ARP.keys())

for key in list_hosts:
    Network.get_instance().remove_host(Network.get_instance().get_host(key))

In [20]:
# Initialize the network and the hosts
clock, controller_host, computing_hosts, q_map = init_network()

2021-07-18 00:56:33,120: Host QPU_0 started processing
2021-07-18 00:56:33,122: Host QPU_1 started processing
2021-07-18 00:56:33,123: Host host_1 started processing


In [21]:
# Run the first ControllerHost communication protocol on a thread
t1 = controller_host.run_protocol(
    controller_host_protocol_preparation_ansatz,
    (q_map, params, terms))

# Run the first ComputingHost communication protocol on a separate thread for each QPU
threads = []
for host in computing_hosts:
    threads.append(host.run_protocol(computing_host_protocol))

# Wait for all threads to finish
t1.join()
for thread in threads:
    thread.join()

2021-07-18 00:56:34,020: host_1 sends BROADCAST message
2021-07-18 00:56:34,120: sending ACK:1 from QPU_1 to host_1
2021-07-18 00:56:34,152: sending ACK:1 from QPU_0 to host_1
2021-07-18 00:56:34,167: host_1 received ACK from QPU_1 with sequence number 0
2021-07-18 00:56:34,173: QPU_1 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_2"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_3"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"

2021-07-18 00:56:34,175: QPU_0 sends CLASSICAL to host_1 with sequence 0
2021-07-18 00:56:34,176: host_1 received ACK from QPU_0 with sequence number 0
2021-07-18 00:56:34,177: QPU_1 awaits classical ACK from host_1 with sequence 0
2021-07-18 00:56:34,179: sending ACK:1 from host_1 to QPU_1
2021-07-18 00:56:34,180: QPU_0 awaits classical ACK from host_1 with sequence 0
2021-07-18 00:56:34,182: host_1 received ACK with sequence number 0
2021-07-18 00:56:34,184: QPU_1 received ACK from host_1 with sequence number 0
2021-07-18 00:56:34,187: sending ACK:1 from host_1 to QPU_0
2021-07-18 00:56:34,188: host_1 received ACK with sequence number 0
2021-07-18 00:56:34,222: QPU_0 received ACK from host_1 with sequence number 0


In [22]:
# Let's see how many ticks we needed for the applying the ansatz
clock.ticks

4

In [23]:
# Repeat what we did above for the second batch of operations

t1 = controller_host.run_protocol(
    controller_host_protocol_expectation_schedule,
    (q_map,))

threads = []

for host in computing_hosts:
    threads.append(host.run_protocol(computing_host_protocol))

t1.join()
for thread in threads:
    thread.join()

2021-07-18 00:56:40,591: host_1 sends BROADCAST message
2021-07-18 00:56:40,910: sending ACK:2 from QPU_0 to host_1
2021-07-18 00:56:40,930: sending ACK:2 from QPU_1 to host_1
2021-07-18 00:56:41,088: QPU_0 received {"QPU_0": [{"name": "REC_HAMILTON", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "hamiltonian": [[-0.04207897647782276, [["Identity", 0]]], [0.17771287465139946, [["PauliZ", 0]]], [0.1777128746513994, [["PauliZ", 1]]], [-0.24274280513140462, [["PauliZ", 2]]], [-0.24274280513140462, [["PauliZ", 3]]], [0.17059738328801052, [["PauliZ", 0], ["PauliZ", 1]]], [0.04475014401535161, [["PauliY", 0], ["PauliX", 1], ["PauliX", 2], ["PauliY", 3]]], [-0.04475014401535161, [["PauliY", 0], ["PauliY", 1], ["PauliX", 2], ["PauliX", 3]]]], "layer_end": 4}], "QPU_1": [{"name": "REC_HAMILTON", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_1"], "pre_allocated_qubits": fal

In [24]:
clock.ticks

6

In [25]:
# Finally collect the results at the ControllerHost
t1 = controller_host.run_protocol(
    controller_host_protocol_expectation_collection,
    (q_map,))

threads = []

for host in computing_hosts:
    threads.append(host.run_protocol(computing_host_protocol_send_results))

t1.join()
for thread in threads:
    thread.join()

2021-07-18 00:56:42,283: host_1 sends BROADCAST message
2021-07-18 00:56:42,403: sending ACK:3 from QPU_0 to host_1
2021-07-18 00:56:42,424: sending ACK:3 from QPU_1 to host_1
2021-07-18 00:56:42,450: QPU_0 received {"QPU_0": [{"name": "SEND_EXP", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 6}], "QPU_1": [{"name": "SEND_EXP", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_1"], "pre_allocated_qubits": false, "layer_end": 6}]} with sequence number 2
2021-07-18 00:56:42,450: QPU_0 sends CLASSICAL to host_1 with sequence 2
2021-07-18 00:56:42,461: QPU_1 received {"QPU_0": [{"name": "SEND_EXP", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 6}], "QPU_1": [{"name": "SEND_EXP", "qids": null, "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_

In [26]:
# We can now aggregate the results from the ComputingHosts by simply summing up all the values.
total_exp = 0
for host in controller_host.computing_host_ids:
    total_exp += controller_host.results[host]['val']
    
total_exp

-0.8074697622991748

That's not very far from PennyLane's simulation value of -0.88179557 Ha! Such a small difference should be reconciled during the optimization process.

In [27]:
clock.ticks

8

## Optimise

Now we can go on to attempt and optimize the parameters until we minimize the expecation value.

Since the scheduler of the controller host has to be run for each submitted set of parameters, it is more efficient to put the whole quantum circuit into one Circuit object.

In [28]:
def vqe_circuit(host, q_map, parameters):
    layers = []
    host_id = list(q_map.keys())
    
    # Initialise the qubits on the computing host
    layers = layers + initialisation_operations(q_map)
    
    # Apply the ansatz
    layers = layers + ansatz_operations(q_map, parameters)
    
    # Send the schedules
    layers = layers + dispatch_hamiltonian_schedules(host, q_map)
    
    # Ask for the values
    layers = layers + ask_for_expectation_values(q_map)
    
    circuit = Circuit(q_map, layers)
    return circuit

In [29]:
def controller_host_protocol_schedules(host, q_map, params, terms):
    host.schedule_expectation_terms(terms, q_map)
    
    monolithic_circuit = vqe_circuit(host, q_map, params)

    host.generate_and_send_schedules(monolithic_circuit)

In [30]:
def controller_host_protocol_get_final_results(host, q_map, params):
    host.receive_results()

In [31]:
def computing_host_protocol_receive_schedules(host):
    host.receive_schedule()

In [32]:
def computing_host_protocol_send_final_results(host):
    host.send_results('expectation')

In [33]:
# Most of the contents of the following function are copied from the demonstration above.

def cost_fn(params):
    params = params.reshape(4, 3)
    
    network = Network.get_instance()
    network.delay = 0
    network.start()

    Clock.reset_clock()

    eqsn = EQSNBackend()

    controller_host = ControllerHost(
        host_id="host_1",
        backend=eqsn
    )

    computing_hosts, q_map = controller_host.create_distributed_network(
        num_computing_hosts=2,
        num_qubits_per_host=4)
    
    controller_host.start()

    network.add_hosts(computing_hosts)
    network.add_hosts([controller_host])
    
    #############################################################
    
    t1 = controller_host.run_protocol(controller_host_protocol_schedules, (q_map, params, terms))
    
    threads = []

    for host in computing_hosts:
        threads.append(host.run_protocol(computing_host_protocol_receive_schedules))

    t1.join()
    for thread in threads:
        thread.join()
        
    #############################################################
    
    t1 = controller_host.run_protocol(controller_host_protocol_get_final_results, (q_map, params))
    
    threads = []

    for host in computing_hosts:
        threads.append(host.run_protocol(computing_host_protocol_send_final_results))

    t1.join()
    for thread in threads:
        thread.join()
        
    #############################################################
    
    for host in computing_hosts:
        network.remove_host(host)
    network.remove_host(controller_host)
    
    total_exp = 0
    for host in controller_host.computing_host_ids:
        total_exp += controller_host.results[host]['val']
    
    return total_exp

In [34]:
np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

params

array([[ 5.54193389,  1.25713095,  3.07479606],
       [ 7.03997361,  5.86710646, -3.07020901],
       [ 2.98479079, -0.47550269, -0.32427159],
       [ 1.28993324,  0.45252622,  4.56873497]])

Let's see if the cost function returns only one final loss value as expected.

In [35]:
cost_fn(params)

2021-06-25 23:45:34,505: Host QPU_0 started processing
2021-06-25 23:45:34,505: Host QPU_1 started processing
2021-06-25 23:45:34,506: Host host_1 started processing
2021-06-25 23:45:34,507: host_1 sends BROADCAST message
2021-06-25 23:45:34,644: sending ACK:1 from QPU_1 to host_1
2021-06-25 23:45:34,644: sending ACK:1 from QPU_0 to host_1
2021-06-25 23:45:34,645: QPU_1 received {"QPU_0": [{"name": "PREPARE_QUBITS", "qids": ["q_0_0", "q_0_1", "q_0_2", "q_0_3"], "cids": null, "gate": null, "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_1"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_0_2"], "cids": null, "gate": "cnot", "gate_param": null, "computing_host_ids": ["QPU_0"], "pre_allocated_qubits": false, "layer_end": 0}, {"name": "TWO_QUBIT", "qids": ["q_0_0", "q_

2021-06-25 23:45:34,681: QPU_0 sends CLASSICAL to host_1 with sequence 0
2021-06-25 23:45:34,684: host_1 received ACK from QPU_1 with sequence number 0
2021-06-25 23:45:34,684: host_1 received ACK from QPU_0 with sequence number 0
2021-06-25 23:45:34,684: QPU_1 awaits classical ACK from host_1 with sequence 0
2021-06-25 23:45:34,686: sending ACK:1 from host_1 to QPU_1
2021-06-25 23:45:34,687: QPU_0 awaits classical ACK from host_1 with sequence 0
2021-06-25 23:45:34,689: sending ACK:1 from host_1 to QPU_0
2021-06-25 23:45:34,691: host_1 received ACK with sequence number 0
2021-06-25 23:45:34,692: QPU_1 received ACK from host_1 with sequence number 0
2021-06-25 23:45:34,695: host_1 received ACK with sequence number 0
2021-06-25 23:45:34,698: QPU_0 received ACK from host_1 with sequence number 0
2021-06-25 23:45:38,804: QPU_0 sends CLASSICAL to host_1 with sequence 1
2021-06-25 23:45:38,805: QPU_1 sends CLASSICAL to host_1 with sequence 1
2021-06-25 23:45:38,805: QPU_0 awaits classical A

-0.8074697622991748

Looks good! Let's disable the `Logger` to avoid clutter during the optimization process.

In [36]:
Logger.DISABLED = True

It should be noted that the networking nature of this simulation makes it tricky to use gradient-based approaches for optimization, as Interlin-q does not support it yet. So we will go for non-gradient based optimizers. 

### SciPy Optimizers

SciPy optimizers from the `minimize` API are too slow for the purely network version, thus they are not included in this notebook.

### Scikit-Quant Optimisers

This library is specialized for non-gradient based optimization of parameterized quantum circuits, and thus is perfect for our needs. Here, the library takes as input `budget` value, which determines how "long" it should attempt minimizing the expectation value.

In [37]:
from skquant.interop.scipy import *

In [38]:
# We need to bound our parameters for the optimization
bounds = np.array([-3*np.pi, 3*np.pi])
bounds = np.tile(bounds, (4*3, 1))

np.random.seed(0)
params = np.random.normal(0, np.pi, (4, 3))

# The library must receive the parameters as a flat 1D array
flattened_parameters = params.flatten()
flattened_parameters

array([ 5.54193389,  1.25713095,  3.07479606,  7.03997361,  5.86710646,
       -3.07020901,  2.98479079, -0.47550269, -0.32427159,  1.28993324,
        0.45252622,  4.56873497])

Let's try the first optimizer.

In [42]:
budget = 100
minimum_bobyqa = minimize(cost_fn, flattened_parameters, method=pybobyqa, bounds=bounds, options={'budget' : budget})

minimum_bobyqa

     fun: -1.1329627749839453
 message: 'completed'
    nfev: 100
  status: 0
 success: True
       x: array([ 6.0388495 , -0.09447084,  2.57788046,  6.54305801,  6.27915045,
       -3.56712461,  2.48787519, -0.2274104 ,  0.28697217,  0.79301763,
       -0.01450685,  5.06565057])

That's very close to PennyLane's -1.13613394 Ha!

Let's try the two other optimizers.

In [43]:
budget = 100
minimum_snobfit = minimize(cost_fn, flattened_parameters, method=snobfit, bounds=bounds, options={'budget' : budget})

minimum_snobfit

     fun: -0.9093011711618758
 message: 'completed'
    nfev: 121
  status: 0
 success: True
       x: array([-7.67176926e-01, -6.06635258e+00,  5.90556587e-01, -4.01872532e-01,
        1.31946891e-03, -3.19481123e+00,  3.95916073e+00, -3.13279619e-01,
        2.32226529e-01, -3.31563689e+00, -5.25544469e+00,  2.24743255e+00])

In [44]:
budget = 100
minimum_imfil = minimize(cost_fn, flattened_parameters, method=imfil, bounds=bounds, options={'budget' : budget})

minimum_imfil

     fun: -1.016230585889092
 message: 'completed'
    nfev: 105
  status: 0
 success: True
       x: array([ 5.54193389,  5.96951993,  5.43099055,  7.03997361,  5.86710646,
        6.35456895,  2.98479079, -0.47550269, -0.32427159,  1.28993324,
        0.45252622,  4.56873497])

It seems that a budget of 100 was not sufficient for them, and thus were not able to reach the same minimum value as that of BOBYQA.